# pypdb demos

This is a set of basic examples of the usage and outputs of the various individual functions included in. There are generally three types of functions.

### Preamble

In [ ]:
%pylab inline
from IPython.display import HTML

# Import from local directory
# import sys
# sys.path.insert(0, '../pypdb')
# from pypdb import *

# Import from installed package
from pypdb import *

%load_ext autoreload
%autoreload 2

# Search functions that return lists of PDB IDs

In [ ]:
found_pdbs = Query(27499440, "PubmedIdQuery").search()
print(found_pdbs[:10])

#### Get a list of PDBs for a specific search term

In [ ]:
found_pdbs = Query("ribosome").search()
print(found_pdbs[:10])

#### Search by PubMed ID Number

In [ ]:
found_pdbs = Query(27499440, "PubmedIdQuery").search()
print(found_pdbs[:10])

#### Search by source organism using NCBI TaxId

In [ ]:
found_pdbs = Query('6239', 'TreeEntityQuery').search() #TaxID for C elegans
print(found_pdbs[:5])

#### Search by a specific experimental method

In [ ]:
found_pdbs = Query('SOLID-STATE NMR', query_type='ExpTypeQuery').search()
print(found_pdbs[:10])

#### Search by protein structure similarity

In [ ]:
found_pdbs = Query('2E8D', query_type="structure").search()
print(found_pdbs[:10])

#### Search by Author

In [ ]:
found_pdbs = Query('Perutz, M.F.', query_type='AdvancedAuthorQuery').search()
print(found_pdbs)

#### Search by organism

In [ ]:
q = Query("Dictyostelium", query_type="OrganismQuery")
print(q.search()[:10])

# Information Search functions
While the basic functions described in the previous section are useful for looking up and manipulating individual unique entries, these functions are intended to be more user-facing: they take search keywords and return lists of authors or dates

#### Find papers for a given keyword

In [ ]:
matching_papers = find_papers('crispr', max_results=10)
print(list(matching_papers)[:10])

# Functions that return information about single PDB IDs

#### Get the full PDB file

In [ ]:
pdb_file = get_pdb_file('4lza', filetype='cif', compression=False)
print(pdb_file[:400])

#### Get a general description of the entry's metadata

In [ ]:
all_info = get_info('4LZA')
print(list(all_info.keys()))

#### Run a Sequence search

Formerly using BLAST, this method now uses MMseqs2

In [ ]:
q = Query("VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTAVAHVDDMPNAL", 
          query_type="sequence", 
          return_type="polymer_entity")

print(q.search())

#### Search by PFAM number

In [ ]:
pfam_info = Query("PF00008", query_type="pfam").search()
print(pfam_info[:5])

# New API for advanced search

The old API will gradually migrate to use these functions

In [ ]:
from pypdb.clients.search.search_client import perform_search
from pypdb.clients.search.search_client import ReturnType
from pypdb.clients.search.operators import text_operators

## Search for all entries that mention the word 'ribosome'

In [ ]:
search_operator = text_operators.DefaultOperator(value="ribosome")
return_type = ReturnType.ENTRY

results = perform_search(search_operator, return_type)

print(results[:10])

## Search for polymers from 'Mus musculus'

In [ ]:
search_operator = text_operators.ExactMatchOperator(value="Mus musculus",
                                                    attribute="rcsb_entity_source_organism.taxonomy_lineage.name")
return_type = ReturnType.POLYMER_ENTITY

results = perform_search(search_operator, return_type)

print(results[:5])

## Search for non-polymers from 'Mus musculus' or 'Homo sapiens'

In [ ]:
search_operator = text_operators.InOperator(values=["Mus musculus", "Homo sapiens"],
                                            attribute="rcsb_entity_source_organism.taxonomy_lineage.name")
return_type = ReturnType.NON_POLYMER_ENTITY

results = perform_search(search_operator, return_type)
print(results[:5])

## Search for polymer instances whose titles contain "actin" or "binding" or "protein"

In [ ]:
search_operator = text_operators.ContainsWordsOperator(value="actin-binding protein",
                                            attribute="struct.title")
return_type = ReturnType.POLYMER_INSTANCE

results = perform_search(search_operator, return_type)

print(results[:5])

## Search for assemblies that contain the words "actin binding protein"
(must be in that order).

For example, "actin-binding protein" and "actin binding protein" will match,
but "protein binding actin" will not.

In [ ]:
search_operator = text_operators.ContainsPhraseOperator(value="actin-binding protein",
                                            attribute="struct.title")
return_type = ReturnType.ASSEMBLY

results = perform_search(search_operator, return_type, verbosity=True)

print(results[:5])

## Search for entries released in 2019 or later

In [ ]:
search_operator = text_operators.ComparisonOperator(
       value="2019-01-01T00:00:00Z",
       attribute="rcsb_accession_info.initial_release_date",
       comparison_type=text_operators.ComparisonType.GREATER)
return_type = ReturnType.ENTRY

results = perform_search(search_operator, return_type)

print(results[:5])

## Search for entries released only in 2019

In [ ]:
search_operator = text_operators.RangeOperator(
    from_value="2019-01-01T00:00:00Z",
    to_value="2020-01-01T00:00:00Z",
    include_lower=True,
    include_upper=False,
    attribute="rcsb_accession_info.initial_release_date")
return_type = ReturnType.ENTRY

results = perform_search(search_operator, return_type)

print(results[:5])

## Search by cell length and suppress query output

In [ ]:
from pypdb.clients.search.search_client import perform_search_with_graph, SearchService, ReturnType
from pypdb.clients.search.operators import text_operators

cell_a_operator = text_operators.RangeOperator(
    attribute='cell.length_a',
    from_value=80,
    to_value=84,
    include_upper=True
)

results = perform_search_with_graph(
    query_object=cell_a_operator,
    return_type=ReturnType.ENTRY,
    verbosity=False,
)

print(results[:5])

## Search for structures under 4 angstroms of resolution

In [ ]:
search_operator = text_operators.ComparisonOperator(
           value=4,
           attribute="rcsb_entry_info.resolution_combined",
           comparison_type=text_operators.ComparisonType.LESS)
return_type = ReturnType.ENTRY

results = perform_search(search_operator, return_type)

print(results[:5])

## Search for structures with a given attribute.

(Admittedly every structure has a release date, but the same logic would
 apply for a more sparse RCSB attribute).


In [ ]:
search_operator = text_operators.ExistsOperator(
    attribute="rcsb_accession_info.initial_release_date")
return_type = ReturnType.ENTRY

results = perform_search(search_operator, return_type)

print(results[:5])

## Search for 'Mus musculus' or 'Homo sapiens' structures after 2019 using graph search


In [ ]:
from pypdb.clients.search.search_client import perform_search_with_graph
from pypdb.clients.search.search_client import ReturnType
from pypdb.clients.search.search_client import QueryGroup, LogicalOperator
from pypdb.clients.search.operators import text_operators

# SearchOperator associated with structures with under 4 Angstroms of resolution
under_4A_resolution_operator = text_operators.ComparisonOperator(
       value=4,
       attribute="rcsb_entry_info.resolution_combined",
       comparison_type=text_operators.ComparisonType.GREATER)

# SearchOperator associated with entities containing 'Mus musculus' lineage
is_mus_operator = text_operators.ExactMatchOperator(
            value="Mus musculus",
            attribute="rcsb_entity_source_organism.taxonomy_lineage.name")

# SearchOperator associated with entities containing 'Homo sapiens' lineage
is_human_operator = text_operators.ExactMatchOperator(
            value="Homo sapiens",
            attribute="rcsb_entity_source_organism.taxonomy_lineage.name")

# QueryGroup associated with being either human or `Mus musculus`
is_human_or_mus_group = QueryGroup(
    queries = [is_mus_operator, is_human_operator],
    logical_operator = LogicalOperator.OR
)

# QueryGroup associated with being ((Human OR Mus) AND (Under 4 Angstroms))
is_under_4A_and_human_or_mus_group = QueryGroup(
    queries = [is_human_or_mus_group, under_4A_resolution_operator],
    logical_operator = LogicalOperator.AND
)

return_type = ReturnType.ENTRY

results = perform_search_with_graph(
  query_object=is_under_4A_and_human_or_mus_group,
  return_type=return_type)
print("\n", results[:10]) # Huzzah